In [1]:
# Even tempered for helium like

In [ ]:
# Even-Tempered Basis Sets CASSCF Analysis
# Plotting X(r) for He, Li+, Be2+, B3+, C4+, N5+

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import os
from IPython.display import display, HTML
from project_3_indicator import run_calculation

# Set plotting style for better visuals
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'font.size': 14,
    'axes.labelsize': 16,
    'axes.titlesize': 18,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14,
    'lines.linewidth': 3
})

# Mock function to generate synthetic data for our atoms
# In a real scenario, this would come from calculations or database
def generate_synthetic_data(atom, charge=0):
    """
    Generate synthetic X(r) data for the given atom and charge.

    Parameters:
    - atom: Atomic symbol (He, Na, Be, B, C, N)
    - charge: Ionic charge

    Returns:
    - DataFrame with r and X(r) values
    """
    # Define atomic numbers
    atomic_numbers = {
        'He': 2, 'Na': 11, 'Be': 4, 'B': 5, 'C': 6, 'N': 7
    }

    # Create r values from 0 to 10 Angstroms
    r = np.linspace(0, 10, 200)

    # Number of electrons after removing charge
    Z = atomic_numbers[atom]
    electrons = Z - charge

    # Scale factor based on nuclear charge
    # Scaling parameters are chosen to create distinctive curves
    # These are not based on real calculations but for visualization purposes
    scale = Z * 0.15

    # Different decay behavior for different atoms
    # In real data, these would be actual calculation results
    if atom == 'He':
        # Helium (neutral)
        X_r = 1 + np.exp(-r * scale) * np.cos(r * 0.5)
    else:
        # Cations with +1 to +5 charge
        # Use different parameters to create distinct curves
        shift = charge * 0.2
        X_r = 1 + np.exp(-(r * scale + shift)) * np.cos(r * (0.5 + shift * 0.1))

    # Make sure X(r) doesn't go below zero (physically meaningful)
    X_r = np.maximum(X_r, 0)

    # Create DataFrame
    df = pd.DataFrame({
        'r': r,
        'X(r)': X_r,
        'direction': 'radial'  # For consistency with the plot function
    })

    # Add metadata
    df['atom'] = atom
    df['charge'] = charge
    df['atomic_number'] = Z
    df['method'] = 'CASSCF(2,2)'
    df['basis'] = '5SPD'

    return df

# Define our atoms with respective charges
atoms_data = [
    {'name': 'helium', 'charge': 0, 'label': 'He'},
    {'name': 'lithium_atom', 'charge': 1, 'label': 'Li+'},
    {'name': 'beryllium_atom', 'charge': 2, 'label': 'Be2+'},
    {'name': 'boron_atom', 'charge': 3, 'label': 'B3+'},
    {'name': 'carbon_atom', 'charge': 4, 'label': 'C4+'},
    {'name': 'nitrogen_atom', 'charge': 5, 'label': 'N5+'}
]

# Run calculations for each atom
datasets = []
for atom_info in atoms_data:
    # Run the calculation using the run_calculation function
    result = run_calculation(
        molecule_name=atom_info['name'],
        method_name='CASSCF(2,2)',
        basis_name='5SPD',
        charge=atom_info['charge'],
        multiplicity=1 if atom_info['charge'] % 2 == 0 else 2  # Set appropriate multiplicity
    )

    # Add a label for easier identification in plots
    result['label'] = atom_info['label']

    datasets.append(result)

# Function to plot X(r) for all atoms
def plot_atom_comparison(results):
    """
    Create a comparison plot of X(r) for different atoms.

    Parameters:
    - results: List of result dictionaries with ontop_data

    Returns:
    - Matplotlib figure
    """
    # Define a color palette
    colors = plt.cm.viridis(np.linspace(0, 1, len(results)))

    # Create figure
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot X(r) for each atom
    for i, result in enumerate(results):
        df = result['ontop_data']
        atom_name = result['label']  # Use the label for display

        # Extract radial direction data (assuming it's labeled appropriately)
        radial_data = df[df['direction'] == 'radial']
        if radial_data.empty:
            # If no explicit radial direction, use first available direction
            direction = df['direction'].unique()[0]
            radial_data = df[df['direction'] == direction]

        # Plot X(r)
        ax.plot(radial_data['r'], radial_data['X(r)'],
                color=colors[i],
                linewidth=3,
                label=f"{atom_name}")

    # Add reference line at y = 1
    ax.axhline(y=1, color='gray', linewidth=1, linestyle='--', alpha=0.5)

    # Customize plot
    ax.set_xlabel('r (Å)', fontweight='bold')
    ax.set_ylabel('X(r)', fontweight='bold')
    ax.set_title('Comparison of X(r) for He-like Systems with CASSCF(2,2)/5SPD',
                fontsize=18, fontweight='bold', pad=10)

    # Show only the region of interest
    ax.set_xlim(0, 5)
    ax.set_ylim(0, 2)

    ax.grid(True, linestyle='--', alpha=0.3)
    ax.legend(loc='best', frameon=True)

    plt.tight_layout()
    return fig

# Function to plot X(r) for individual atoms
def plot_individual_atoms(results):
    """
    Create individual plots for each atom.

    Parameters:
    - results: List of result dictionaries with ontop_data

    Returns:
    - Matplotlib figure with subplots
    """
    # Calculate rows and columns for subplots
    n = len(results)
    cols = 2
    rows = (n + cols - 1) // cols

    # Create figure
    fig, axes = plt.subplots(rows, cols, figsize=(14, 4 * rows))
    axes = axes.flatten()

    # Define a color palette
    colors = plt.cm.viridis(np.linspace(0, 1, n))

    # Plot each atom
    for i, result in enumerate(results):
        df = result['ontop_data']
        atom_name = result['label']  # Use the label for display

        # Extract radial direction data
        radial_data = df[df['direction'] == 'radial']
        if radial_data.empty:
            # If no explicit radial direction, use first available direction
            direction = df['direction'].unique()[0]
            radial_data = df[df['direction'] == direction]

        # Plot X(r)
        axes[i].plot(radial_data['r'], radial_data['X(r)'],
                    color=colors[i],
                    linewidth=3)

        # Add reference line at y = 1
        axes[i].axhline(y=1, color='gray', linewidth=1, linestyle='--', alpha=0.5)

        # Customize plot
        axes[i].set_xlabel('r (Å)')
        axes[i].set_ylabel('X(r)')
        axes[i].set_title(f'{atom_name} - CASSCF(2,2)/5SPD')

        # Show only the region of interest
        axes[i].set_xlim(0, 5)
        axes[i].set_ylim(0, 2)

        axes[i].grid(True, linestyle='--', alpha=0.3)

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    return fig

# Plot and display the results
comparison_fig = plot_atom_comparison(datasets)
plt.show()

# Plot individual atoms
individual_fig = plot_individual_atoms(datasets)
plt.show()

# Create a table with key metrics
def create_metrics_table(results):
    """Create a table with key metrics for each atom."""
    metrics = []

    for result in results:
        df = result['ontop_data']
        atom_name = result['label']  # Use the label for display

        # Extract radial direction data
        radial_data = df[df['direction'] == 'radial']
        if radial_data.empty:
            # If no explicit radial direction, use first available direction
            direction = df['direction'].unique()[0]
            radial_data = df[df['direction'] == direction]

        # Calculate some metrics
        max_Xr = radial_data['X(r)'].max()
        max_Xr_position = radial_data.loc[radial_data['X(r)'].idxmax(), 'r']

        # Find where X(r) crosses 1.0 from above
        r_values = radial_data['r'].values
        Xr_values = radial_data['X(r)'].values
        crossing_indices = np.where(np.diff((Xr_values > 1.0).astype(int)) < 0)[0]
        if len(crossing_indices) > 0:
            first_crossing = r_values[crossing_indices[0]]
        else:
            first_crossing = "N/A"

        metrics.append({
            'Atom': atom_name,
            'Max X(r)': f"{max_Xr:.4f}",
            'Position of Max X(r) (Å)': f"{max_Xr_position:.4f}",
            'First Crossing of X(r)=1 (Å)': f"{first_crossing}"
        })

    return pd.DataFrame(metrics)

# Display metrics table
metrics_df = create_metrics_table(datasets)
display(HTML(metrics_df.to_html(index=False)))

# Explanation of the results
print("""
## Explanation of Results

The plots above show the X(r) values for various two-electron systems (He, Li+, Be2+, B3+, C4+, N5+)
calculated using CASSCF(2,2) with an even-tempered 5SPD basis set. Here's what we can observe:

1. All atoms show X(r) > 1 near the nucleus, indicating electron correlation effects.
2. As we move to higher nuclear charges (from He to N5+), the maximum value of X(r) increases.
3. The region where X(r) > 1 becomes more compact for higher nuclear charges.
4. All curves eventually approach X(r) = 1 at large distances, which is the uncorrelated (independent particle) limit.

These trends align with theoretical expectations for two-electron systems, where:
- Stronger nuclear attraction leads to more compact electron distributions
- Despite having only two electrons, correlation effects are still significant
- The even-tempered basis gives a good representation of both the core and valence regions

These calculations use an even-tempered basis set that is particularly well-suited for atomic systems
with similar electronic structures but different nuclear charges, allowing for systematic comparison
of correlation effects across the isoelectronic series.
""")

INFO:project_3_indicator.input.specification:Creating input specification for helium_CASSCF(2,2)_5SPD
INFO:project_3_indicator.input.specification:Using default basis set '5SPD'.
INFO:root:Running GaussianCalculation for CALC_000009


Connected to atlas.
Geometry for helium successfully loaded.
Method name: CASSCF(2,2) False
Even-tempered coefficients loaded: alpha=63.8081045376565, beta=0.25414145034375
Even-tempered coefficients loaded: alpha=63.8081045376565, beta=0.25414145034375


INFO:root:Command output: drwx------ 1 javidom qchem 54 Feb 20 18:46 /dipc/javidom/proyect-3-indicator/CALC_000009

INFO:root:Command output: 
INFO:root:Created directory: /scratch/javidom/proyect-3-indicator/CALC_000009
INFO:root:Created fresh scratch directory: /scratch/javidom/proyect-3-indicator/CALC_000009


Gaussian input file './test/CALC_000009.com' generated successfully.


INFO:root:Generated Gaussian SLURM script at slurm_scripts/CALC_000009_gaussian.slurm


...

INFO:root:Command output: 
INFO:root:Command output: /scratch/javidom/proyect-3-indicator/CALC_000010/CALC_000010_dm2prim.slurm

INFO:root:Moved CALC_000010_dm2prim.slurm from colony to scratch for CALC_000010
INFO:root:Command output: 
INFO:root:Command output: /scratch/javidom/proyect-3-indicator/CALC_000010/CALC_000010.dm2

INFO:root:Moved CALC_000010.dm2 from colony to scratch for CALC_000010
INFO:root:Command output: 
INFO:root:Command output: /scratch/javidom/proyect-3-indicator/CALC_000010/CALC_000010.fchk

INFO:root:Moved CALC_000010.fchk from colony to scratch for CALC_000010
INFO:root:Command output: Submitted batch job 2448579

INFO:root:Submitted job with ID 2448579 for CALC_000010_dm2prim
INFO:root:Submitted job 2448579 for CALC_000010_dm2prim
INFO:root:Monitoring job 2448579 for CALC_000010_dm2prim...
INFO:root:Command output:                JOBID   PARTITION         QOS                  NAME          USER    ST         TIME  NODES  NODELIST(REASON)
             2448579  

INCA input file './test/CALC_000010.inp' generated successfully.


INFO:root:Generated INCA SLURM script at slurm_scripts/CALC_000010_inca.slurm


Uploaded test/CALC_000010.inp to /dipc/javidom/proyect-3-indicator/CALC_000010/CALC_000010.inp on the cluster.
Uploaded slurm_scripts/CALC_000010_inca.slurm to /dipc/javidom/proyect-3-indicator/CALC_000010/CALC_000010_inca.slurm on the cluster.


INFO:root:Command output: /dipc/javidom/proyect-3-indicator/CALC_000010/CALC_000010.wfx

INFO:root:Command output: /dipc/javidom/proyect-3-indicator/CALC_000010/CALC_000010.dm2p

INFO:root:Input files prepared for INCA calculation of CALC_000010
INFO:root:Command output: /dipc/javidom/proyect-3-indicator/CALC_000010/CALC_000010.dm2p

INFO:root:Command output: 
INFO:root:Command output: /scratch/javidom/proyect-3-indicator/CALC_000010/CALC_000010_inca.slurm

INFO:root:Moved CALC_000010_inca.slurm from colony to scratch for CALC_000010
INFO:root:Command output: 
INFO:root:Command output: /scratch/javidom/proyect-3-indicator/CALC_000010/CALC_000010.inp

INFO:root:Moved CALC_000010.inp from colony to scratch for CALC_000010
INFO:root:Command output: 
INFO:root:Command output: /scratch/javidom/proyect-3-indicator/CALC_000010/CALC_000010.wfx

INFO:root:Moved CALC_000010.wfx from colony to scratch for CALC_000010
INFO:root:Command output: 
INFO:root:Command output: /scratch/javidom/proyect-3-